In [1]:
import random

# Algoritmos de Euclides

Antes de empezar necesitaremos un sistema para cálculo del máximo común divisor (mcd en castellano, gcd en inglés) de dos números, y el inverso de un número en un anillo cíclico. Ambas cosas se conocen desde hace tiempo: son dos "algoritmos de Euclides"

Algoritmo de Euclides para determinar el máximo común divisor (gcd) de dos enteros a yb

In [2]:
def gcd(a, b):
    while b != 0:
        a, b = b, a % b
    return a

print('gcd(2, 3) = ', gcd(2, 3))
print('gcd(20, 30) = ', gcd(20, 30))
print('gcd(50720, 48184) = ', gcd(50720, 48184))


gcd(2, 3) =  1
gcd(20, 30) =  10
gcd(50720, 48184) =  2536


Algoritmo generalizado de Euclides para encontrar el inverso multiplicativos de un número en un anillo cíclico $\mathbb{Z}_{\phi}$

In [3]:
def multiplicative_inverse(e, phi):
    d = 0
    x1 = 0
    x2 = 1
    y1 = 1
    temp_phi = phi
    
    while e > 0:
        temp1 = temp_phi // e
        temp2 = temp_phi - temp1 * e
        temp_phi = e
        e = temp2
        
        x = x2 - temp1 * x1
        y = d - temp1 * y1
        
        x2 = x1
        x1 = x
        d = y1
        y1 = y

    if temp_phi == 1:
        return d + phi
    # no inverse: return None
    return None

print('3^{-1} mod 10 = ', multiplicative_inverse(3, 10))
print('2^{-1} mod 10 = ', multiplicative_inverse(2, 10))
print('25^{-1} mod 119 = ', multiplicative_inverse(25, 119))

3^{-1} mod 10 =  7
2^{-1} mod 10 =  None
25^{-1} mod 119 =  100


In [4]:
def is_prime(num):
    if num == 2:
        return True
    if num < 2 or num % 2 == 0:
        return False
    for n in range(3, int(num**0.5)+2, 2):
        if num % n == 0:
            return False
    return True

for i in [2, 5, 19, 25, 222, 314, 317]:
    print(f'{i}: ', is_prime(i))

2:  True
5:  True
19:  True
25:  False
222:  False
314:  False
317:  True


# RSA

RSA son unas pocas funciones sencillas:

- Generación de claves
- Cifrado y descifrado son iguales (y simplemente es una potencia)

In [5]:
def generate_keypair(p, q):
    if not (is_prime(p) and is_prime(q)):
        raise ValueError('Both numbers must be prime.')
    elif p == q:
        raise ValueError('p and q cannot be equal')
    #n = pq
    n = p * q

    #Phi is the totient of n
    phi = (p - 1) * (q - 1)

    # Choose an integer e such that e and phi(n) are coprime
    e = random.randrange(1, phi)

    # Use Euclid's Algorithm to verify that e and phi(n) are coprime
    g = gcd(e, phi)
    while g != 1:
        e = random.randrange(1, phi)
        g = gcd(e, phi)

    #Use Extended Euclid's Algorithm to generate the private key
    d = multiplicative_inverse(e, phi)
    
    #Return public and private keypair
    #Public key is (e, n) and private key is (d, n)
    return ((e, n), (d, n))

def encrypt(pk, number):
    # Unpack the key into it's components
    key, n = pk
    return (number ** key) % n

decrypt = encrypt

In [6]:
pk, sk = generate_keypair(17, 23)
print(f'Publickey (e, n): {pk} Private-key (d, n): {sk}')

Publickey (e, n): (109, 391) Private-key (d, n): (197, 391)


Fíjate: si generamos otro par de claves, aunque usemos los mismos primos, obtendremos unas claves diferentes. Eso es porque el parámetro $e$ se escoge al azar

In [7]:
pk, sk = generate_keypair(17, 23)
print(f'Publickey: {pk} Private-key: {sk}')

Publickey: (317, 391) Private-key: (181, 391)


Vamos a intentar cifrar un texto sencillo:

In [8]:
print(encrypt(pk, 'hola'))

TypeError: unsupported operand type(s) for ** or pow(): 'str' and 'int'

No podemos: RSA solo puede cifrar enteros. Una posibilidad es codificar el mensaje como un conjunto de enteros

In [9]:
print([encrypt(pk, ord(c)) for c in 'hola'])

[372, 263, 146, 218]


¿Qué pasa si intentamos cifrar varias veces lo mismo?

In [10]:
print([encrypt(pk, ord(c)) for c in 'aaaa'])

[218, 218, 218, 218]


Pocas veces querremos eso. RSA debe usarse siguiendo recomendaciones como PKCS#1

# (semi) Homorfismo

RSA es semihomomórfico con la multiplicación: se pueden hacer cálculos con los números cifrados, aunque no sepas lo que son ni qué resultado tienes. Al descifrar, el resultado es correcto.

Por ejemplo, vamos a multiplicar los mensajes cifrados c1 y c2, que son los cifrados de 5 y 2 respectivamente

In [11]:
m1 = 5
c1 = encrypt(pk, m1)
print(f'encrypt(pk, {m1}) = {c1}')
print(f'decrypt(sk, {c1}) = {decrypt(sk, c1)}')


encrypt(pk, 5) = 241
decrypt(sk, 241) = 5


In [12]:
m2 = 2
c2 = encrypt(pk, m2)
print(f'encrypt(pk, {m2}) = {c2}')
print(f'decrypt(sk, {c2}) = {decrypt(sk, c2)}')

encrypt(pk, 2) = 236
decrypt(sk, 236) = 2


In [13]:
cm = c1 * c2
print(f"c1 = {c1}; c2 = {c2}; cm = {cm}")

c1 = 241; c2 = 236; cm = 56876


Un atacante no sabe cuánto vale c1 ni c2, ni sabe qué valor tiene cm, pero sabe que, sea lo que sea, ha multiplicado c1 y c2 y cuando se descifre el resultado va a ser correcto

In [14]:
print(f'decrypt(sk, c1 * c2) = m1 * m2 = {m1} * {m2} = {decrypt(sk, cm)}')

decrypt(sk, c1 * c2) = m1 * m2 = 5 * 2 = 10


Según la utilidad, el semihomorfismo puede ser útil o no:

- Sistemas PET (private enhanced technologies) necesitas calcular sin descifrar. Por ejemplo, voto electrónico
- Pero en general no querremos que un atacante pueda multiplicar una orden de pago por otro número y que el resultado sea válido: recomendaciones PKCS#1

# PyCryptoDome

La función de arriba solo sirve para ver cómo funciona RSA a alto nivel. Veamos ahora cómo de grandes son los números involucrados en estos cifrados. Ojo: ¡mide cuánto tiempo necesitamos para generar las claves!

In [15]:
# Clave de 2048 bits
from Crypto.PublicKey import RSA
key2048 = RSA.generate(2048)
key2048

RsaKey(n=23542220262221135616479100277313313630398447228011535524355261249556050320357446526199986671623317083015079297435395105318406319798285043202615732844604669012772888224996385954876260616814458993540889067041712057508582987594294192257619868896218760956265305186773335938633837822955878861748265321377224012340766704186061648292220754031533679429871963254204719040307990190761876427269492545273003298567735640008189806818265147872704346458892248805289560824583055798955352206016125425960454101290377403869198950673992163578904588982969295028654089046022630879449599759805035480844756855425421517041147305379459069078967, e=65537, d=3576756445228433515926612470226111415198702092700472392938421597903925920316753819390165361389037767910968530212921388889564241973717505762675204112695556546075951722786960265997877946223073503497057119497906937403496663067830191682700169897914921385500307364461328805727712943881649844003043743304591596973830465386208881148575554805740620443981548631532788564

In [16]:
key4096 = RSA.generate(4096)
key4096

RsaKey(n=9811142028917447223373588614482178642992005682365223413635573155208949776709842594310798807936061561917613966840530306007657199648928441519912234744577533315320057570979650349462312316821958362266491686650228435043395863990076204486304081672966152017507129171328027415977507427753957038997484963513690092115207099049896864942997073997928970307852341215703337405848871113859302512528089369122210405594915702031199814691450414800002024443501449587405062095382760194325060225629903723766086302677563020109284167154188012754539772772049142963829429659804371616704441610765929792605316042510705275804272569028837531068469136133984620518380802166585024415324632234167325791751111621499862936493222953300626423032263314756389930323451177987721286077006036970124132200007577042549371192958588165130514634012941181652290339470326434119039782428146610481325412286472367441254652546072806212443779578777544917497068901480440899503106475828640771492412341264493549379960454703359604483155903585206327782

# Ejercicios

Hemos visto cómo crear claves con PyCryptoDrome, pero no cómo usarlo para cifrar o descifrar.

Recuerda de las transparencias que no es recomendable utilizar RSA "de forma pura", es decir, sin tener en cuenta muchas consideraciones sobre padding, conversiones, longitudes... que se recogen en [PKCS#1](https://en.wikipedia.org/wiki/PKCS_1). De hecho, PyCryptoDome no nos va a dejar utilizar el cifrado y descifrado directamente.

Observa que la línea siguiente da un error, avisando que uses el módulo  `Crypto.Cipher.PKCS1_OAEP`

In [19]:
key2048.encrypt(b'hola', None)

NotImplementedError: Use module Crypto.Cipher.PKCS1_OAEP instead

**Aunque no se debe**, vamos a utilizar la función `_encrypt()`, que no está documentada pero la puedes encontrar en el código: https://github.com/Legrandin/pycryptodome/blob/master/lib/Crypto/PublicKey/RSA.py#L147

In [18]:
c = key2048._encrypt(15)
d = key2048._decrypt(c)
print(f"Cifrado: {c}")
print(f"Descifrado: {d}")

Cifrado: 10283644035387406808090093039379336170236811409579305519990899975943492837953240810693478045731816113599744677338249835608819177402646578683657880000245032029824733824053379706225221449749089351992202068237017268982431780082529701497492940112802734570593977889830136302202367862567040155970733883651667901995332250788449987995771894091253725069719180239209062109690441973737726864948350166705667163379780216433298345886352896831909803940088273159646544162997678475140330129508335990967468467048730943124935320865613135393861906374548234032551806888147949495767548265549131518317881335880307148866336588943391188471731
Descifrado: 15


Usando estas funciones `_encrypt()` y `_decrypt()` para cifrar cadenas:

1. Una posibilidad es cifrar cada caracter por separado y cifrarlos también por separado, como hemos hecho antes. ¿Cuándo ocupa el cifrado, en bytes?
1. Otra posibilidad es codificar la cadena como un enorme entero, es decir, cada caracter representa un byte de un número entero: `msg = int.from_bytes(b"hola mundo", "big")` ¿Cuánto ocupa el cifrado, en bytes?
1. ¿Puedes probar el método anterior para cifrar una cadena realmente larga, como `msg = int.from_bytes(b"hola mundo" * 1000, "big")` ? ¿Por qué crees que no funciona? ¿Cómo lo harías?

Vamos a hacer las cosas bien: cifra `"hola mundo"` y `"hola mundo" * 1000` usando PKCS1. Encontrarás en ejemplo en la documentación de pyCryptoDome: https://pycryptodome.readthedocs.io/en/latest/src/cipher/oaep.html

## Cifrado híbrido

En el tema de TLS veremos un cifrado híbrido: ciframos con RSA la clave AES que usamos para cifrar el texto.
1. Bob: Crea par de claves RSA
1. Alice: Crea clave simétrica AES. Cifra la clave AES con la clave pública de Bob. Envía mensaje
1. Alice: cifra "hola mundo" con clave AES. Envía mensaje
1. Bob: descifra clave AES con clave privada. Descifra mensaje de Alice

Entre los ejemplos de RSA precisamente verás algo así: https://pycryptodome.readthedocs.io/en/latest/src/examples.html#encrypt-data-with-rsa

- ¿Puedes hacer cifrado híbrido del mensaje "hola mundo"?
- ¿Se te ocurre por qué es necesario el cifrado híbrido?